In [4]:
import requests
import json
import time

TEST_FENS = [
    # Starting position
    "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1",
    # Queen sacrifice position (Black to move)
    "1b1r1kr1/pp1bqppp/1n2p3/2p1n3/3PpPN1/2P1P2P/PPN1Q1P1/3BRKR1 b - - 0 1",
    # Invalid FEN (will be skipped by the API)
    "r1bq1rk1/1pp1bpp1/2np1n1p/p3p3/P3P3/2NP1N1P/1PPBBPP1/R2Q1RK1 b - a3 0 1", 
    # Complex middle game position (White to move)
    "r2q1rk1/pppbbpp1/2np1n1p/4p3/4P3/2NP1N1P/PPPBBPP1/R2Q1RK1 w - - 0 1",
]

start_time = time.time()
API_URL = "http://127.0.0.1:9999/analyze"
payload = {
    "fens": TEST_FENS,
    "nodes_limit": 50000,      # Max 50,000 nodes if time limit is not met
}

response = requests.post(
        API_URL, 
        headers={"Content-Type": "application/json"}, 
        data=json.dumps(payload)
    )
    
end_time = time.time()

# Check for success
response.raise_for_status() 

# Parse the JSON response
analysis_results = response.json()

In [ ]:
analysis_results